In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# 키워드 추출

### 전처리

In [ ]:
df_clustered = pd.read_csv("/content/drive/MyDrive/미래내일 일경험/df_2.csv")
df_clustered = df_clustered.sort_values(by='cluster_num', ascending=True)
df_clustered['cluster_num'] = range(len(df_clustered))

In [ ]:
len(df_clustered)

54

In [ ]:
import re

def clean_text(text):
  text = re.sub('(<([^>]+)>)', '', text) # 괄호와 괄호 안 내용 제거
  text = text.replace("&#039","").replace("&quot;","")

  text = re.sub(r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' , '', text) # 이메일 제거

  text = re.sub(r'(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text) # url 제거

  text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ]+)', '', text) # 자음, 모음 만 존재시 제거

  text = re.sub(r'<[^>]*>', '', text) # 괄호 <> 와 괄호내 문자제거

  text = re.sub(r'\([^)]*\)', '', text) # 괄호 () 와 괄호내 문자제거

  text = re.sub(r'\[[^)]*\]', '', text) # 괄호 [] 와 괄호내 문자제거

  text = re.sub(r'[0-9]', '', text)  # 숫자 제거

  text = re.sub(r'[^\w\s.]', '', text) #단어, 띄어쓰기, 문자"." 이외의 특수문자 모두 제거

  text = text.strip()
  text = " ".join(text.split())

  text = re.sub(r'^\w{,6}[\s=]?[가-힣]{3}\s?기자[\s=]{,2}', '', text)

  return text

In [ ]:
df_clustered['title']=df_clustered['title'].apply(clean_text)
df_clustered['cleaned_description'] = df_clustered['description'].apply(clean_text)

### 키워드 추출

In [ ]:
!pip install keybert
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 29.5 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
from keybert import KeyBERT

okt = Okt()

# 모델1
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

# 모델2
# key_model = KeyBERT('paraphrase-multilingual-mpnet-base-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 토큰화
def tokenize(text):
  tokens = okt.morphs(text)
  return ' '.join(tokens)

df_clustered['tokenized_description'] = df_clustered['cleaned_description'].apply(tokenize)

,tokenized_description
0,현대차 는 국내외 기업 및 기관 에서 표준 으로 사용 하고 있는 플라스틱 RF 카드...
1,톱 다운 램프 슬래브 구축 공법 서대문 재개발 현장 도입 공기 단축 공 사비 절감 ...
2,바이오 이노 베이 터 시나몬 랩 이 스낵 형 건강 기능 식품 의 특허 를 취득 했다...
3,휴 온스 그룹 헬 스케 어 부자 재 전문 기업 휴엠앤씨 는 니트로 사 민 을 억제 ...
4,특히 키즈 라인 에는 궁 중 비책 의 특허 출원 성분 인 로얄 테라 티 가드 를 함...
5,생활 문화 기업 오 쎄가 천연 성분 인 LDSF complex 의 특허 를 출원 했...
6,의료 인공 지능 기업 제이 엘 케이 는 자사 가 개발 한 AI 학습 원천 기술 의 ...
7,비만 치료 제 시장 이 오는 년 억 달러 규모 로 성장 이 예상 되는 가운데 지난해...
8,홍익대학교 이형주 교수 가 농구 지도 에 있어 체계 적 인 교육 방법 을 제공 하기...
9,주식회사 페스티벌 온 은 지난 일 특허청 으로부터 지식 재산 경 영 인증 을 취득 ...


In [ ]:
# 불용어 지정
with open('/content/drive/MyDrive/미래내일 일경험/stopword.txt') as fin:
  stopwords = fin.readlines()
  for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].replace('\n','')

new_stopwords = stopwords + ['각','이','가','에','는','그리고','하지만','그','은','을','했','하면서',
             '에게','있','이','의','하','한','다','과','때문','또한','는데','선보이며'
             '가','등','들','등','거나','으로부터', '과도','한다고','하기로','한다','이르면',
             '하기로', '돕는다', '지난해', '지난','관련','지난달','한다고','알리는','동일하게','사랑스러운', '건기']

# 불용어 제거 함수
def remove_stopwords(text):
    # 텍스트를 공백으로 분리하여 토큰화
    tokens = text.split()
    # 불용어 목록에 있는 단어를 제외한 나머지 단어만 남김
    tokens = [word for word in tokens if word not in stopwords]
    # 다시 공백으로 연결하여 텍스트로 반환
    return ' '.join(tokens)

df_clustered['tokenized_description_removed'] = df_clustered['tokenized_description'].apply(remove_stopwords)

['않다', '되어다', '되다', '하다', '어떻다', '이렇다', '이다', '어제', '매일', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에

In [ ]:
## 시간은 total 2분 이내

# 키워드 추출
df_clustered['keyword'] = df_clustered['tokenized_description_removed'].apply(
    lambda x: kw_model.extract_keywords(x, keyphrase_ngram_range=(1,2), stop_words=new_stopwords, top_n=3)
)

df_clustered['keyword'] = df_clustered['keyword'].apply(lambda x: ', '.join([kw for kw, _ in x]))
news_df = df_clustered

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '만큼', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '어찌됏든', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-package

# 유사도 추출


### 특허 전처리

In [ ]:
# 전처리된 특허 데이터셋 파일
patent_df = pd.read_csv('/content/drive/MyDrive/미래내일 일경험/patent_df.csv')[['No.', '발명의 명칭', '요약', 'LINK']]

In [ ]:
# 중복 제거
patent_df = patent_df.drop_duplicates(subset=['발명의 명칭'])

# 텍스트 정제
patent_df['cleaned_발명의_명칭']=patent_df['발명의 명칭'].apply(clean_text)
patent_df['cleaned_요약']=patent_df['요약'].apply(clean_text)



### Kobert

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-9qdbh20t/kobert-tokenizer_6211eb22d45246e293a96bcf5c0ea33b
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-9qdbh20t/kobert-tokenizer_6211eb22d45246e293a96bcf5c0ea33b
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=8747c46f5f8dc60048ed71300013a87134451bd2a115f05abe96152d5bde82b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ge868coc/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', last_hidden_states=True)
model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Bert 클래스 정의
class Bert(nn.Module):
    def __init__(self, model_name, dropout_p=0.25):
        super(Bert, self).__init__()
        self.bertmodel = BertModel.from_pretrained(model_name, output_attentions=True, output_hidden_states=True)
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, input_ids, attention_mask):
        outputs = self.bertmodel(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰의 임베딩
        pooled_output = self.dropout(pooled_output)  # 선택적으로 드롭아웃 적용
        return pooled_output


In [ ]:
# 모델 초기화
model = Bert(
    model_name='skt/kobert-base-v1',
    dropout_p=0.25,
)

In [ ]:
import torch

# 뉴스 데이터와 특허 데이터를 임베딩하는 함수 정의
def embed_texts(texts, batch_size=16, max_length=512):
    # 시퀀스의 최대 길이를 512로 제한하고, 배치 내 시퀀스의 길이를 맞추기 위해 패딩을 사용
    input_encodings = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    input_ids = input_encodings['input_ids']
    attention_mask = input_encodings['attention_mask']

    embeddings = []
    model.eval()  # 평가 모드
    with torch.no_grad():  # 그래디언트를 계산하지 않도록 설정
      for i in range(0, len(input_ids), batch_size):
            batch_input_ids = input_ids[i:i+batch_size]
            batch_attention_mask = attention_mask[i:i+batch_size]
            batch_embeddings = model(batch_input_ids, batch_attention_mask)
            embeddings.append(batch_embeddings.cpu().numpy())

    return np.concatenate(embeddings, axis=0)

In [ ]:
## 시간: 30분.. 소요..
news_embeddings = embed_texts(news_df['cleaned_description'].tolist())
patent_embeddings = embed_texts(patent_df['cleaned_요약'].tolist())


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# # 각 뉴스 기사에 대해 모든 특허와의 유사도 계산
# top_n = 3 # 각 뉴스 기사에 대해 추천할 상위 특허 수
# for i, news_embedding in enumerate(news_embeddings):
#     similarities = cosine_similarity(news_embedding.reshape(1, -1), patent_embeddings)
#     top_indices = similarities.argsort()[0][-top_n:][::-1]  # 상위 N개의 특허 선택
#     print(f"뉴스 기사 {i}와 유사한 상위 {top_n}개의 특허 요약:")

#     for idx in top_indices:
#         print(f"특허 요약: {patent_df.iloc[idx]['요약']}")
#         print(f"유사도: {similarities[0][idx]:.4f}")
#     print("\n")


뉴스 기사 0와 유사한 상위 3개의 특허 요약:
특허 요약: 센싱 값에 따라 풍속이 자동 조절되는 선풍기는, 선풍기 회전 날개의 회전 속도를 조절하는 선풍기 구동부와; 선풍기 회전 날개의 방향을 조절하는 방향 조절부와; 주변 온도를 감지하는 온도 센서와; 주변에 사람이 있는지를 확인하기 위한 움직임 센서와; 각 센서의 센싱 값을 입력 받아 선풍기의 구동부와 방향 조절부를 제어하는 동작 제어부;를 포함하는 것을 특징으로 한다.
유사도: 0.9638
특허 요약: 본 발명은 자율주행을 위해 구축된 네트워크의 보안성이 향상될 수 있도록, 자율주행 과정에서 데이터를 생성하는 장비에 대한 보안성 평가 및 분석을 수행하는 차량 네트워크 데이터 관리 시스템에 관한 것이다.
유사도: 0.9475
특허 요약: 본 발명은 분사, 숏팅, 코팅, 도장, 타설, 포설, 주입, 포장, 다짐, 양생, 정화, 청정, 청소, 작업, 면처리, 페인팅, 크리닝, 공기청정, 공기정화 등을 하는 드론, 장비, 장치, 방법, 숏팅기, 코팅기, 도장기, 포장기 등에 관한 것이다. 
유사도: 0.9465


뉴스 기사 1와 유사한 상위 3개의 특허 요약:
특허 요약: 본 발명은 중견업체 이상의 기업들과 같이 스마트팩토리 구현을 위해 솔루션 전체를 구축하는 대규모 작업 대신, 공정별로 모듈 단위로 미니 스마트팩토리를 도입할 수 있어 소규모 제조기업에 맞춤형 솔루션을 제공할 수 있는 모듈 단위로 도입 가능한 소규모 제조업용 클라우드 기반 스마트팩토리 솔루션 제공 시스템에 관한 것이다.
유사도: 0.9552
특허 요약: 응급상태의 자율주행 차량 원격제어 시스템 및 방법은 자율주행 차량의 주행데이터와 주변 데이터를 모니터링하고, 차량 상태를 파악하여 자율주행이 정상적으로 수행될 수 없는 응급상태인 경우, 자율주행차량을 원격 제어함으로써, 교통 흐름을 방해하는 요인을 제거하여 자율주행으로 인한 사고를 예방한다. 또한, 실시예에서는 서버와 차량간 데이터 통신 효율을 향상시켜 원격제어 안전성을 높임으로써, 자율주행 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 유사도 계산 및 추천
top_n = 3 # 각 뉴스 기사에 대해 추천할 상위 특허 수
recommendations = []

for news_index, news_embedding in enumerate(news_embeddings):
    news_embedding = news_embedding.reshape(1, -1)

    # 각 뉴스 기사에 대해 모든 특허와의 유사도 계산
    similarities = cosine_similarity(news_embedding, patent_embeddings).flatten()

    # 유사도가 높은 상위 N개의 특허 선택
    top_patents = np.argsort(similarities)[-top_n:][::-1]  # 유사도가 높은 순으로 정렬
    recommended_patents = patent_df.iloc[top_patents][['요약', 'LINK', '발명의 명칭']].values

    # 결과 저장
    for i in range(top_n):
        recommendations.append({
            'news_index': news_index,
            'news_title': news_df.iloc[news_index]['top_title'],
            'news_description': news_df.iloc[news_index]['description'],
            'news_keyword': news_df.iloc[news_index]['keyword'],
            'news_link': news_df.iloc[news_index]['link'],
            'recommended_patent_summary': recommended_patents[i, 0],
            'patent_link': recommended_patents[i, 1],
            'patent_title': recommended_patents[i, 2],
            'similarity_score': similarities[top_patents[i]]  # 유사도 점수 추가
        })

# 추천 결과를 데이터프레임으로 저장
recommendations_df = pd.DataFrame(recommendations)

# 결과 출력
print(recommendations_df[['news_index', 'news_title', 'news_description', 'news_keyword', 'news_link',
                          'patent_title', 'recommended_patent_summary', 'patent_link', 'similarity_score']])


     news_index                                     news_title  \
0             0                현대차 직원은 출입증도 달라…전자잉크 LCD 사원증 특허   
1             0                현대차 직원은 출입증도 달라…전자잉크 LCD 사원증 특허   
2             0                현대차 직원은 출입증도 달라…전자잉크 LCD 사원증 특허   
3             1                  반도건설, 특허출원 지하 구조물 시공법 현장 첫 적용   
4             1                  반도건설, 특허출원 지하 구조물 시공법 현장 첫 적용   
..          ...                                            ...   
157          52  [기업]글로벌 회원제 온라인 쇼핑몰 뉴트론, 총 28억까지 출산 지원금 혜택 제공   
158          52  [기업]글로벌 회원제 온라인 쇼핑몰 뉴트론, 총 28억까지 출산 지원금 혜택 제공   
159          53            이뮤노바이옴, 특허 담은 '아이엠바이오틱스 피부&장건강' 선보여   
160          53            이뮤노바이옴, 특허 담은 '아이엠바이오틱스 피부&장건강' 선보여   
161          53            이뮤노바이옴, 특허 담은 '아이엠바이오틱스 피부&장건강' 선보여   

                                      news_description  \
0    현대차는 국내외 기업 및 기관에서 표준으로 사용하고 있는 플라스틱 RF 카드 대신 ...   
1    현대차는 국내외 기업 및 기관에서 표준으로 사용하고 있는 플라스틱 RF 카드 대신 ...   
2    현대차는 국내외 기업 및 기관에서 표준으로 사용하고

In [ ]:
# recommendations_df.to_csv("/content/drive/MyDrive/미래내일 일경험/recommendations_df_kobert_2.csv")